In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from transformers import AutoModelForImageSegmentation

c:\Users\Сергей\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pip install kornia

Note: you may need to restart the kernel to use updated packages.


In [3]:
model = AutoModelForImageSegmentation.from_pretrained('briaai/RMBG-2.0', trust_remote_code=True)
torch.set_float32_matmul_precision(['high', 'highest'][0])
model.to('cuda')
model.eval()

c:\Users\Сергей\AppData\Local\Programs\Python\Python312\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\Сергей\AppData\Local\Programs\Python\Python312\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


BiRefNet(
  (bb): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=192, out_features=576, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=192, out_features=192, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=192, out_features=768, bias=T

In [4]:
# Data settings
image_size = (1024, 1024)
transform_image = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
import os
import re

for _, dirs, _ in os.walk('C:/Users/Сергей/Desktop/cmpolik/models'):
  for dir in dirs:
   #saveing files
   match = re.search(r'\d+', dir)
   dir_num = int(match.group())
   os.mkdir(f'C:/Users/Сергей/Desktop/cmpolik/Final_Dataset/{dir_num}')
   for _, _, files in os.walk(f'C:/Users/Сергей/Desktop/cmpolik/models/{dir}'):
     for file in files:
       if file.startswith('Camera') and file.endswith('.png'):
        # turn to 3 chanel
        image = Image.open(f'C:/Users/Сергей/Desktop/cmpolik/models/{dir}/{file}')
        image = image.convert("RGB")
        input_images = transform_image(image).unsqueeze(0).to('cuda')

        # Prediction
        print(f"{dir}/{file}")
        with torch.no_grad():
            preds = model(input_images)[-1].sigmoid().cpu()
        print("Mujic done")
        pred = preds[0].squeeze()
        pred_pil = transforms.ToPILImage()(pred)
        mask = pred_pil.resize(image.size)
        image.putalpha(mask)
        
        if match:
            new_file = file.replace("Camera_", "")
            image.save(f'C:/Users/Сергей/Desktop/cmpolik/Final_Dataset/{dir_num}/{new_file}')
    


mvslc1/Camera_back.png
Mujic done
mvslc1/Camera_front.png
Mujic done
mvslc1/Camera_side.png
Mujic done


FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: 'C:/Users/Сергей/Desktop/cmpolik/Final_Dataset/1'